In [3]:
import json
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from sklearn.naive_bayes import BernoulliNB,MultinomialNB
from sklearn.cross_validation import train_test_split
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
import nltk

# read data

In [4]:
with open('Book_reviews50000.json') as data_file:  
    data = []
    for line in data_file:
        data.append(json.loads(line))
        

In [5]:
df = pd.DataFrame(data)

In [24]:
text = df['reviewText']

In [25]:
def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

In [26]:
y = df['overall']
y = y.map(partition)

# make a balanced subsample 

In [37]:
indices = y[y == 'negative'].index
neg_indices = y[y == 'negative'].index
random_indices1 = np.random.choice(indices, 5000, replace=False)
random_indices = np.random.choice(neg_indices, 5000, replace=False)
healthy_sample = df.loc[random_indices]

KeyError: 'None of [[ 2901 26287  8650 ..., 11336  1902  1636]] are in the [columns]'

In [29]:
X_train, X_test, y_train, y_test = train_test_split(text, y, test_size=0.2, random_state=1)

# Simple model

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(X_train).toarray()
test_vector = vectorizer.transform(X_test).toarray()

In [ ]:
clf = BernoulliNB()
clf.fit(vectors, y_train)
clf.score(test_vector, y_test)

In [31]:
clf = MultinomialNB()
clf.fit(vectors, y_train)
clf.score(test_vector, y_test)

0.94499999999999995

In [32]:
clf = LogisticRegression(C=1e5)
clf.fit(vectors, y_train)
clf.score(test_vector, y_test)

0.93999999999999995

# Stemming , Stop words removal , Lowering , Tokenization ,Pruning (numbers and punctuation)

In [8]:
def tokenize(text):
    PUNCTUATION = set(string.punctuation)
    STOPWORDS = set(stopwords.words('english'))
    STEMMER = PorterStemmer()
    tokens = word_tokenize(text)
    lowercased = [t.lower() for t in tokens]
    no_punctuation = []
    for word in lowercased:
        punct_removed = ''.join([letter for letter in word if not letter in PUNCTUATION])
        no_punctuation.append(punct_removed)
    no_stopwords = [w for w in no_punctuation if not w in STOPWORDS]
    stemmed = [STEMMER.stem(w) for w in no_stopwords]
    return str([w for w in stemmed if w])

In [9]:
tokenize(text[0])

"[u'spiritu', u'mental', u'inspir', u'book', u'allow', u'question', u'moral', u'help', u'discov', u'realli']"

# naive bayes and logistic model

In [10]:
stem_vectorizer = TfidfVectorizer(stop_words='english',tokenizer=tokenize)
stem_vectors = stem_vectorizer.fit_transform(X_train).toarray()
stem_test_vectors =stem_vectorizer.transform(X_test).toarray()

In [11]:
clf = BernoulliNB()
clf.fit(stem_vectors, y_train)
clf.score(stem_test_vectors, y_test)

0.877

In [12]:
clf = MultinomialNB()
clf.fit(stem_vectors, y_train)
clf.score(stem_test_vectors, y_test)

0.89939999999999998

In [13]:
clf = LogisticRegression(C=1e5)
clf.fit(stem_vectors, y_train)
clf.score(stem_test_vectors, y_test)

0.8992